In [1]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import pandas as pd
import re

!pip install transformers
from transformers import pipeline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import re 
import string 
import time
pd.set_option("display.max_rows",50)
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate

# machine learning model for email classification
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.8 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# imap setup

In [2]:
# account credentials
username = "Your_email_address"
password = "Your_Password"
# use your email provider's IMAP server, you can look for your provider's IMAP server on Google
# or check this page: https://www.systoolsgroup.com/imap/
# for office 365, it's this:
imap_server = "imap-mail.outlook.com"

def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

In [3]:
# create an IMAP4 class with SSL 
imap = imaplib.IMAP4_SSL(imap_server)
# authenticate
imap.login(username, password)

('OK', [b'LOGIN completed.'])

In [4]:
status, messages = imap.select("INBOX")
# number of top emails to fetch
#N = int(messages[0])
N = int(messages[0])
# total number of emails
messages = int(messages[0])

# List containing subject, from, and body
emailsList = []

# Loop to read emails

In [5]:
def parse_uid(msg_data):
    #print(type(msg_data))
    pattern_uid = re.compile(r'\d+ \(UID (?P<uid>\d+)\)')
    match = pattern_uid.match(msg_data)
    return match.group('uid')

In [6]:
def get_uid_and_rfc822(folder):
    imap.select(folder)
    raw_emails = []

    result, data = imap.uid('search', None, "ALL")
    ids = data[0]
    id_list = ids.split()

    for i in id_list:
        res, msg = imap.uid('fetch', i, '(RFC822)')
        #raw_email = msg[0][1]
        #raw_emails.append(raw_email)
        for response in msg:
                #rfc_string = response[0].decode()
                #print(type(rfc_string))
                if isinstance(response, tuple):

                    # parse a bytes email into a message object
                    msg = email.message_from_bytes(response[1])
                    # decode the email subject
                    subject, encoding = decode_header(msg["Subject"])[0]
                    if isinstance(subject, bytes):
                        # if it's a bytes, decode to str
                        subject = subject.decode(encoding)
                    # decode email sender
                    From, encoding = decode_header(msg.get("From"))[0]
                    if isinstance(From, bytes):
                        From = From.decode(encoding)
                    print("Subject:", subject)
                    print("From:", From)
                    # if the email message is multipart
                    if msg.is_multipart():
                        # iterate over email parts
                        for part in msg.walk():
                            # extract content type of email
                            content_type = part.get_content_type()
                            content_disposition = str(part.get("Content-Disposition"))
                            try:
                                # get the email body
                                body = part.get_payload(decode=True).decode()
                            except:
                                pass
                            if content_type == "text/plain" and "attachment" not in content_disposition:
                                # print text/plain emails and skip attachments
                                #print("This is the plain body: " + body)

                                #print("Whats happening to the body??: " + body)
                                emailGroup = [i, subject, From, body]
                                print('EMAIL GROUP', emailGroup)
                                print(type(emailsList))
                                emailsList.append(emailGroup)
                            elif "attachment" in content_disposition:
                                print('sad')
                                # download attachment
                                filename = part.get_filename()
                                if filename:
                                    folder_name = clean(subject)
                                    if not os.path.isdir(folder_name):
                                        # make a folder for this email (named after the subject)
                                        os.mkdir(folder_name)
                                    filepath = os.path.join(folder_name, filename)
                                    # download attachment and save it
                                    open(filepath, "wb").write(part.get_payload(decode=True))
                    else:
                        # extract content type of email
                        content_type = msg.get_content_type()
                        # get the email body
                        body = msg.get_payload(decode=True).decode()
                        if content_type == "text/plain":
                            # print only text email parts
                            print(body)
                    print("="*100)

            #return id_list, raw_emails
            

In [7]:
get_uid_and_rfc822('Inbox')

Subject: Upcoming Event
From: Stephen L <stephen.lanna1994@gmail.com>
EMAIL GROUP [b'162', 'Upcoming Event', 'Stephen L <stephen.lanna1994@gmail.com>', "I am so excited for the upcoming event - it's going to be a blast!\r\n"]
<class 'list'>
Subject: Bad Service >:(
From: Stephen L <stephen.lanna1994@gmail.com>
EMAIL GROUP [b'164', 'Bad Service >:(', 'Stephen L <stephen.lanna1994@gmail.com>', 'I am so disappointed with the service I received - I will never be returning\r\nhere again.\r\n']
<class 'list'>
Subject: Thank you
From: Stephen L <stephen.lanna1994@gmail.com>
EMAIL GROUP [b'166', 'Thank you', 'Stephen L <stephen.lanna1994@gmail.com>', 'I am so grateful for your help and support - thank you so much.\r\n']
<class 'list'>
Subject: Frustrated
From: Stephen L <stephen.lanna1994@gmail.com>
EMAIL GROUP [b'168', 'Frustrated', 'Stephen L <stephen.lanna1994@gmail.com>', "I am so frustrated with the wait time - I'm going to take my business\r\nelsewhere.\r\n"]
<class 'list'>
Subject: Love

In [8]:
imap.list()

('OK',
 [b'(\\HasNoChildren) "/" Archive',
  b'(\\HasChildren \\Trash) "/" Deleted',
  b'(\\HasNoChildren) "/" Deleted/Negative',
  b'(\\Marked \\HasNoChildren) "/" "Deleted/Negative (1)"',
  b'(\\HasNoChildren) "/" Deleted/Positive',
  b'(\\HasNoChildren) "/" "Deleted/Positive (1)"',
  b'(\\HasNoChildren \\Drafts) "/" Drafts',
  b'(\\Marked \\HasNoChildren) "/" Inbox',
  b'(\\HasNoChildren \\Junk) "/" Junk',
  b'(\\Marked \\HasNoChildren) "/" Negative',
  b'(\\Marked \\HasNoChildren) "/" "Negative (1)"',
  b'(\\HasNoChildren) "/" Notes',
  b'(\\HasNoChildren) "/" Outbox',
  b'(\\Marked \\HasNoChildren) "/" Positive',
  b'(\\HasNoChildren) "/" "Positive (1)"',
  b'(\\HasNoChildren \\Sent) "/" Sent',
  b'(\\Marked \\HasNoChildren) "/" testing',
  b'(\\HasNoChildren) "/" ToClassify'])

In [9]:
imap.create('Negative')
imap.create('Positive')
imap.create('testing')
imap.list()

('OK',
 [b'(\\HasNoChildren) "/" Archive',
  b'(\\HasChildren \\Trash) "/" Deleted',
  b'(\\HasNoChildren) "/" Deleted/Negative',
  b'(\\Marked \\HasNoChildren) "/" "Deleted/Negative (1)"',
  b'(\\HasNoChildren) "/" Deleted/Positive',
  b'(\\HasNoChildren) "/" "Deleted/Positive (1)"',
  b'(\\HasNoChildren \\Drafts) "/" Drafts',
  b'(\\Marked \\HasNoChildren) "/" Inbox',
  b'(\\HasNoChildren \\Junk) "/" Junk',
  b'(\\Marked \\HasNoChildren) "/" Negative',
  b'(\\Marked \\HasNoChildren) "/" "Negative (1)"',
  b'(\\HasNoChildren) "/" Notes',
  b'(\\HasNoChildren) "/" Outbox',
  b'(\\Marked \\HasNoChildren) "/" Positive',
  b'(\\HasNoChildren) "/" "Positive (1)"',
  b'(\\HasNoChildren \\Sent) "/" Sent',
  b'(\\Marked \\HasNoChildren) "/" testing',
  b'(\\HasNoChildren) "/" ToClassify'])

# Create DataFrame to hold emails

In [10]:
emailDf = pd.DataFrame(emailsList, columns=['UID', 'Subject', 'From', 'Body'])

In [11]:
emailDf.head()

,UID,Subject,From,Body
0,b'162',Upcoming Event,Stephen L <stephen.lanna1994@gmail.com>,I am so excited for the upcoming event - it's ...
1,b'164',Bad Service >:(,Stephen L <stephen.lanna1994@gmail.com>,I am so disappointed with the service I receiv...
2,b'166',Thank you,Stephen L <stephen.lanna1994@gmail.com>,I am so grateful for your help and support - t...
3,b'168',Frustrated,Stephen L <stephen.lanna1994@gmail.com>,I am so frustrated with the wait time - I'm go...
4,b'170',Love ur products,Stephen L <stephen.lanna1994@gmail.com>,I am so pleased with the product I received - ...


# Setup pipeline for sentiment analysis

In [12]:
sentiment_pipeline = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


# Gather email contents for Sentiment Analysis

In [13]:
# Function to extract contents from DataFrame
def get_email_contents(emailDf):
  uids = []
  subjects = []
  bodies = []
  senders = []

  for i in range(len(emailDf)):
    row = emailDf.iloc[i]
    uids.append(row['UID'])
    subjects.append(row['Subject'])
    bodies.append(row['Body'])
    senders.append(row['From'])
  return uids, subjects, bodies, senders

In [14]:
uids, subjects, bodies, senders = get_email_contents(emailDf)

In [15]:
def remove_newlines(list_of_strings):
  new_list = []
  for string in list_of_strings:
    new_list.append(string.replace('\n', '').replace('\r', ''))
  return new_list

In [16]:
bodies = remove_newlines(bodies)

In [17]:
def reduce_list_size(list_of_strings):
    # Create an empty list to hold the reduced strings
    reduced_list = []
    # Iterate through the list of strings
    for string in list_of_strings:
        # Split the string into its tokens
        tokens = string.split()
        # If the length of the tokens is greater than 512, reduce it to 512
        if len(tokens) > 512:
            tokens = tokens[:512]
            # Join the tokens to make the string
            reduced_string = ' '.join(tokens)
            # Append the reduced string to the reduced list
            reduced_list.append(reduced_string)
        # If the length of the tokens is less than 512, append the string to the reduced list
        else:
            reduced_list.append(string)
    # Return the reduced list
    return reduced_list

In [18]:
bodies = reduce_list_size(bodies)

# Perform Sentiment Analysis on Emails

In [19]:
# Performing sentiment analysis on only subjects

# Output sentiment classifications as binary value, 0 as negative, 1 as positive
def sentiment_one_only(uids, contents, email_type):
  binary_value = {}
  print("Analyzing sentiment on email %s..."%email_type)
  #results = sentiment_pipeline(contents)
  #print("Analysis COMPLETE.")
  #print("")
  i = 0
  for string in contents:
    print("%s: "%email_type, string)
    results = sentiment_pipeline(string[:512])
    print(f"Sentiment Classification: {results[0].get('label')}, with score: {results[0].get('score')}")
    
    if results[0].get('label') == 'NEGATIVE':
      binary_value[uids[i]] = 0
    else:
      binary_value[uids[i]] = 1

    i += 1

  return binary_value

# Sentiment analysis on titles only

In [20]:
subject_sents = sentiment_one_only(uids, subjects, "subject")
print(subject_sents)

Analyzing sentiment on email subject...
subject:  Upcoming Event
Sentiment Classification: POSITIVE, with score: 0.9987686276435852
subject:  Bad Service >:(
Sentiment Classification: NEGATIVE, with score: 0.9997867941856384
subject:  Thank you
Sentiment Classification: POSITIVE, with score: 0.9998352527618408
subject:  Frustrated
Sentiment Classification: NEGATIVE, with score: 0.9997798800468445
subject:  Love ur products
Sentiment Classification: POSITIVE, with score: 0.9898982644081116
subject:  I'm pissed
Sentiment Classification: NEGATIVE, with score: 0.999605119228363
subject:  Color me impressed
Sentiment Classification: POSITIVE, with score: 0.9998779296875
subject:  I'm not satisfied
Sentiment Classification: NEGATIVE, with score: 0.9997848868370056
subject:  Yay
Sentiment Classification: POSITIVE, with score: 0.9140976071357727
subject:  We're through
Sentiment Classification: POSITIVE, with score: 0.9833534955978394
{b'162': 1, b'164': 0, b'166': 1, b'168': 0, b'170': 1, b'1

# Sentiment analysis on bodies only

In [21]:
body_sents = sentiment_one_only(uids, bodies, "body")
print(body_sents)

Analyzing sentiment on email body...
body:  I am so excited for the upcoming event - it's going to be a blast!
Sentiment Classification: POSITIVE, with score: 0.9998310804367065
body:  I am so disappointed with the service I received - I will never be returninghere again.
Sentiment Classification: NEGATIVE, with score: 0.9996556043624878
body:  I am so grateful for your help and support - thank you so much.
Sentiment Classification: POSITIVE, with score: 0.9998295307159424
body:  I am so frustrated with the wait time - I'm going to take my businesselsewhere.
Sentiment Classification: NEGATIVE, with score: 0.9988405108451843
body:  I am so pleased with the product I received - I will definitely be orderingmore in the future.
Sentiment Classification: POSITIVE, with score: 0.9997034668922424
body:  I am so angry with the customer service I received - I will be filing acomplaint.
Sentiment Classification: NEGATIVE, with score: 0.9996387958526611
body:  I am so impressed with the quality o

# Filter emails into Positive or Negative folders

In [22]:
def parse_uid(msg_data):
    pattern_uid = re.compile(r'\d+ \(UID (?P<uid>\d+)\)')
    match = pattern_uid.match(msg_data)
    return match.group('uid')

In [23]:
# Function to sort emails into Positive or Negative folders by sentiment
# Takes dictionary with UID as key, binary value representing sentiment as value
def sort_by_sentiment(sentiments):
  for uid in sentiments.keys():
    if sentiments.get(uid) == 0:
      result = imap.uid('COPY', uid, 'Negative')
    else:
      result = imap.uid('COPY', uid, 'Positive')

    if result[0] == 'OK':
      mov, data = imap.uid('STORE', uid , '+FLAGS', '(\Deleted)')
      imap.expunge()

In [24]:
# Classification on email bodies gave us a higher score, lets use that
sort_by_sentiment(body_sents)